In [15]:
run CELL.ipynb

In [ ]:
#these are the parameters we must specify
ionThresh = 0
openThresh = [2,15]
numCells = 7
ionChans = [12,1]

#these are the parameters that have preset values
#can still change if desired
#C = [25, 150, 25]
#G = [0.133,0.015,0.133]
#N = [2,12,2]
#gapJunction = 1
#theFilter = 0.01
#NiGAP = 80
#NiMARG = 4200
#pokeHoleMARG = False

#we want to vary gapJunction, so we'll substitute in the values
#in the list below
gJs = [0.0001,0.001,0.01,0.1,1]

rateMultiplier = {'0.2':37.5,'0.01':15000}

In [ ]:
#In the code below, we fit the depolarization times to get a speed
#in terms of cells per timestep (the variable labeled theRate).
#We want to convert this to centimeters per second; so we multiply
#theRate by the following:

# ~~~~ (CGLeng)*(1/T)  ~~~~

#where CGLeng is the combined lengths of a cell and a gap between cells,
#and T is the length of a timestep.

#CGLeng's formula is just 

# ~~~~ CGLeng = C[1]+G[1] ~~~~

#the length of the cell plus the length of the gap between the cells. The
#formula for T is given through the diffusion equation, that is

# ~~~~ T = l^2/(2*n*D) ~~~~

#where n is the dimension, so n = 3, and D is the diffusion coefficient,
#so D = 1.334*10^(-5). l^2 is the mean squared displacement, and we will
#write it as 

# ~~~~ l^2 = (<length of block>)^2*(<probability it moves to a new block>) + 0^2*(<probability it doesn't move to a new block>) ~~~~

#which can be rewritten as 

# ~~~~ l^2 = (C[0]/N[0])^2*theFilter*(1-<prob. it chooses to not move if it passes the filter>) ~~~~

#where C[0]/N[0] is the length of a side of an INTRA block (and since they should all be equal 
#you could use C[1]/N[1] or C[2]/N[2] instead) and theFilter is the variable for the filtration rate
#as set above. To get the probability that it stays, first recall that we are assuming that the ion is
#in an INTRA block surrounded on each of its six sides by another INTRA block, and that there are no other
#ions. We need to compute the "move scores" for each of the six adjacent blocks, as well as the "move score"
#for the ion's current block. Then the probability it stays is the "move score" for the current block divided
#by the sum of all the "move scores":

# ~~~~ <prob. it doesn't move to a new block> = <stay movescore> / (<stay movescore>+6*<adjacent block movescore>) ~~~~

#The adjacent block move score is the surface area of the face between blocks divided by a function of the number of ions
#in the destination block:

# ~~~~ <adjacent block move score> = (C[0]/N[0])^2/(1+e^(0-ionThresh)) ~~~~

#note that again since we assume all block lengths are equal, we could use C[0]*C[1] or C[2]^2 or something like that
#and get the same answer. Also note that the reason the 0 is there is because there are 0 ions in the destination square.
#The stay move score is similar, we just use the cube root of the volume  (so a side length) divided by square root of the 
#total surface in the numerator:

# ~~~~ <stay move score> = (C[0]/N[0])/sqrt(6*(C[0]/N[0])^2)) / (1+e^(0-ionThresh)) ~~~~

#I think that in the denominator the ion that's moving is included with the total number of ions, but we can
#discount it as insignificant and leave the zero there.

#So, putting it all together, 

blockLength = C[0]/N[0]

stayMoveScore = (blockLength/np.sqrt(6*blockLength**2))/(1+np.exp(-ionThresh))

adjBlockMoveScore = blockLength**2/(1+np.exp(-ionThresh))

lSquared = blockLength**2*theFilter*(stayMoveScore/(stayMoveScore+6*adjBlockMoveScore))

T = lSquared/(6*1.334*10**(-5))

rateMultiplier = (C[1]+G[1])/T

#we have that the rate conversion to centimeters per second is rateMultiplier, so use this instead.

In [ ]:
rates = []

for gJ in gJs:
    myModel = createModel(numCells, ionThresh, openThresh, ionChans, gapJunct = gJ)
    depol = runModel(myModel)

    fitcoeffs = np.round(np.polyfit(range(2,len(depol)), depol[1:-1], 1),3)
    thePoly = np.poly1d(fitcoeffs)
    theRate =  1/fitcoeffs[0]

    rates.append(theRate*rateMultiplier)

gJs = [0.1,1]
rates = [2,4]
figl = plt.figure()
axl = figl.add_axes([0,0,1,1])
axl.set_xlabel('This is the x-axis (Gap Junction Coefficients')
axl.set_ylabel('This is the y-axis (Conduction Velocity (cm/s))')
axl.set_title('This is the Title')
axl.scatter(gJs, rates, label = 'Can Label Like This')
axl.plot(gJs, rates)
axl.legend()
figl.savefig('GraphTitle.png',bbox_inches='tight')

forpd = {}
forpd['Parameter We Are Varying (Gap Junction Coefficients'] = gJs  
forpd['Conduction Velocities'] = rates
df = pd.DataFrame(data = forpd)
df.to_csv('TitleofDataFrame.csv',index = False)